In [5]:
from dotenv import load_dotenv
import os

load_dotenv()
WATSONX_API_KEY = os.environ["WATSONX_API_KEY"]
WATSONX_SPACE_ID = os.environ["WATSONX_SPACE_ID"]
WATSONX_DEPLOYMENT_ID = os.environ["WATSONX_DEPLOYMENT_ID"]

## CALLING METHOD 1 directly

In [11]:
import requests

# NOTE: you must manually set API_KEY below using information retrieved from your IBM Cloud account (https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-authentication.html?context=wx)
token_response = requests.post('https://iam.cloud.ibm.com/identity/token', data={"apikey":
 WATSONX_API_KEY, "grant_type": 'urn:ibm:params:oauth:grant-type:apikey'})
mltoken = token_response.json()["access_token"]

header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}


payload_scoring = {
    "messages": [
        {
            "role": "user",
            "content": "การลามีกี่ประเภท"
        }
    ]
}

response_scoring = requests.post(f'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/{WATSONX_DEPLOYMENT_ID}/ai_service?version=2021-05-01', json=payload_scoring,
 headers={'Authorization': 'Bearer ' + mltoken})

print("Scoring response")
try:
    print(response_scoring.json())
except ValueError:
    print(response_scoring.text)
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Scoring response
{'choices': [{'finish_reason': 'stop', 'index': 0, 'message': {'content': 'ตามนโยบายการลาของบริษัท มีการระบุประเภทของการลาที่ได้รับการอนุมัติไว้ดังนี้:\n\n1. การลาชั่วคราว\n2. การลาที่ได้รับ\n3. การลาคลอด\n4. การลาพักผ่อนสำหรับบิดา\n5. การลาโดยไม่ได้รับค่าจ้าง\n6. วันหยุดชดเชย\n\nนอกจากนี้ ยังมีการระบุประเภทของการลาในลักษณะอื่นๆ ดังนี้:\n\n- การลาประจำปีที่ได้รับค่าจ้าง\n- การลาโดยไม่ได้รับค่าจ้าง\n\nดังนั้น กรณีทั้งหมดมีการกล่าวถึงการลา 8 ประเภท', 'role': 'assistant'}}], 'created': 1747636374, 'created_at': '2025-05-19T06:32:58.014Z', 'id': 'chatcmpl-7475a0d94200fcc34f394ed1ad9c031d', 'model': 'mistralai/mistral-medium-2505', 'model_id': 'mistralai/mistral-medium-2505', 'model_version': '1.0.0', 'object': 'chat.completion', 'system': {'warnings': [{'id': 'disclaimer_warning', 'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in t

In [12]:
print(response_scoring.json()["choices"][0]["message"]["content"])

ตามนโยบายการลาของบริษัท มีการระบุประเภทของการลาที่ได้รับการอนุมัติไว้ดังนี้:

1. การลาชั่วคราว
2. การลาที่ได้รับ
3. การลาคลอด
4. การลาพักผ่อนสำหรับบิดา
5. การลาโดยไม่ได้รับค่าจ้าง
6. วันหยุดชดเชย

นอกจากนี้ ยังมีการระบุประเภทของการลาในลักษณะอื่นๆ ดังนี้:

- การลาประจำปีที่ได้รับค่าจ้าง
- การลาโดยไม่ได้รับค่าจ้าง

ดังนั้น กรณีทั้งหมดมีการกล่าวถึงการลา 8 ประเภท


## CALLING METHOD 2 using SDK

In [13]:
import os
from ibm_watsonx_ai import APIClient, Credentials
import getpass

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=WATSONX_API_KEY
)

client = APIClient(credentials)

client.set.default_space(WATSONX_SPACE_ID)

'SUCCESS'

In [17]:
# deployment_id
# messages = []
remote_question = "วันลามีกี่ประเภท"
# messages = [{"role":"user","content":"พนักงานเข้าทำงานกี่โมง"}]
messages = [{ "role" : "user", "content": "hi" }, { "role" : "assistant", "content": "I'm good!" }, { "role" : "user", "content": remote_question }]
payload = { "messages": messages }
result = client.deployments.run_ai_service("2a0b107a-6786-4838-99e2-3db3f3700424", payload)
if "error" in result:
    print(result["error"])
else:
    print(result)

{'choices': [{'finish_reason': 'stop', 'index': 0, 'message': {'content': 'According to the provided document, leave can be categorized into 7 types. These are:\n\n1. Sick leave\n2. Maternity leave\n3. Personal leave\n4. Military service leave\n5. Leave due to congenital disability\n6. Training leave\n7. Ordination leave\n\nTherefore, there are 7 types of leave.', 'role': 'assistant'}}], 'created': 1747636515, 'created_at': '2025-05-19T06:35:16.928Z', 'id': 'chatcmpl-b8312831a73f4823a19437483688f8cf', 'model': 'mistralai/mistral-medium-2505', 'model_id': 'mistralai/mistral-medium-2505', 'model_version': '1.0.0', 'object': 'chat.completion', 'system': {'warnings': [{'id': 'disclaimer_warning', 'message': 'This model is a Non-IBM Product governed by a third-party license that may impose use restrictions and other obligations. By using this model you agree to its terms as identified in the following URL.', 'more_info': 'https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-m

In [18]:
print(result["choices"][0]["message"]["content"])

According to the provided document, leave can be categorized into 7 types. These are:

1. Sick leave
2. Maternity leave
3. Personal leave
4. Military service leave
5. Leave due to congenital disability
6. Training leave
7. Ordination leave

Therefore, there are 7 types of leave.
